In [12]:
import torch
import torch.nn as nn
import transformers
from transformers import AutoModel, BertTokenizerFast
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
import pandas as pd
import numpy as np
import nltk
import warnings
import string
import re
warnings.filterwarnings('ignore')
pd.set_option('max_colwidth',None)

In [3]:
import torch, os
import pandas as pd
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast
from torch.utils.data import Dataset

2023-12-17 12:16:52.338995: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cpu'

In [46]:
data = pd.read_csv('/home/pradeepdhanawade/Pradeep_Personal/Evolent/train.csv',sep='\t',names=['class','text'],header=None)
# data.head(2)
data = data[:10000]
# data.size

In [47]:
class_map = {1:0,2:1,3:2,4:3,5:4}
data['class'] = data['class'].map(class_map)

In [48]:
data['class'].unique()

array([3, 4, 1, 0, 2])

In [49]:
data_test = pd.read_csv('/home/pradeepdhanawade/Pradeep_Personal/Evolent/test.csv',sep='\t',names=['text'],header=None)
data_test.head(2)
data_test = data_test[:100]
# data.size

In [50]:
x = data['text']
y = data['class']

In [51]:
y = y.astype('int')

In [52]:
x_test = data_test['text']

In [53]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [54]:
bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [55]:
tokens_train = tokenizer.batch_encode_plus(x.to_list(),padding=True,truncation=True)

In [56]:
tokens_test = tokenizer.batch_encode_plus(x_test.to_list(), padding=True,truncation=True)

In [57]:
train_sequence = torch.tensor(tokens_train['input_ids'])
train_mask_sequence = torch.tensor(tokens_train['attention_mask'])
train_label_sequence = torch.tensor(y.to_list())

In [58]:
test_sequence = torch.tensor(tokens_test['input_ids'])
test_mask_sequence = torch.tensor(tokens_test['attention_mask'])

In [59]:
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler,Dataset

In [60]:
batch_size = 32
# wrap tensors
train_data = TensorDataset(train_sequence,train_mask_sequence,train_label_sequence)
# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

# **Model** Architecture

In [61]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [62]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
        super(BERT_Arch, self).__init__()

        self.bert = bert

        self.dropout = nn.Dropout(0.1)

        self.relu =  nn.ReLU()

        self.fc1 = nn.Linear(768,512)

        self.fc2 = nn.Linear(512,5)



    def forward(self, sent_id, mask):

        #pass the inputs to the model
        _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)

        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

        x = self.fc2(x)

        return x

In [63]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

In [64]:
# model = model.to(device)
model

BERT_Arch(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

In [65]:
from transformers import AdamW

optimizer = AdamW(model.parameters(),lr = 1e-5)

In [66]:
np.unique(y)

array([0, 1, 2, 3, 4])

In [67]:
from sklearn.utils.class_weight import compute_class_weight
#compute the class weights
# class_weights = compute_class_weight(class_weight='balanced',classes= np.unique(y),y= y)
class_weights = compute_class_weight(class_weight='balanced',classes= np.unique(y),y= y)
print('Class Weights:',class_weights)

# converting list of class weights to a tensor
weights= torch.tensor(class_weights,dtype=torch.float)
# print('weights: ',weights)
# push to GPU
# weights = weights.to(device)

# define the loss function
# cross_entropy  = nn.NLLLoss(weight=weights)
cross_entropy = nn.CrossEntropyLoss(weight=weights)
print('type of cross entropy:', type(cross_entropy))
# print('entropy value:', cross_entropy)
# number of training epochs
epochs = 1

Class Weights: [0.90826521 1.89753321 1.48809524 0.93940817 0.61143381]
type of cross entropy: <class 'torch.nn.modules.loss.CrossEntropyLoss'>


# Fine Tuning

In [68]:
# function to train the model
def train():
    model.train()
    total_loss, total_accuracy = 0, 0

    # empty list to save model predictions
    total_preds=[]

    # iterate over batches
    for step,batch in enumerate(train_dataloader):

        # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # push the batch to gpu
#         batch = [r.to(device) for r in batch]

        sent_id, mask, labels = batch
#         print('sent_id: ', sent_id)
#         print('mask: ', mask)
#         print('labels: ', labels)

        # clear previously calculated gradients
        model.zero_grad()

        # get model predictions for the current batch
        preds = model(sent_id, mask)
#         print('preds: ', preds)
#         print('pred size: ',preds.shape)
#         print('label size: ', labels.shape)
        
        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)
        print('loss: ',loss)
#         break
        # add on to the total loss
        total_loss = total_loss + loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

      # predictions are in the form of (no. of batches, size of batch, no. of classes).
      # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss, total_preds

## Train the model

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')
preds = model(test_sequence, test_mask_sequence)
#defining epochs
epochs = 1

# empty lists to store training and validation loss of each epoch
# train_losses=[]
# valid_losses=[]

#for each epoch
for epoch in range(epochs):

    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    #train model
    train_loss, _ = train()

    #evaluate model
    # valid_loss, _ = evaluate()

    #save the best model
    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     torch.save(model.state_dict(), 'saved_weights.pt')
    torch.save(model.state_dict(), 'bertgenai.pt')
    # append training and validation loss
    # train_losses.append(train_loss)
    # valid_losses.append(valid_loss)

    # print(f'\nTraining Loss: {train_loss:.3f}')
    # print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 1
loss:  tensor(1.5952, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5678, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5904, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6025, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6500, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5880, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6399, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5898, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6331, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6165, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5757, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6449, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5900, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6091, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6484, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6017, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5855, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6405, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6088, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6355, grad_fn=<NllLo

loss:  tensor(1.5971, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6093, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6219, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6017, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6114, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5963, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5958, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5967, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5966, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6132, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6052, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6030, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5940, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5998, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6106, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5980, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6108, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6147, grad_fn=<NllLossBackward0>)
loss:  tensor(1.5748, grad_fn=<NllLossBackward0>)
loss:  tensor(1.6018, grad_fn=<NllLossBackward0>)


In [1]:
# model.push_to_hub("aipradeepd/Evolent_BERT_Classifier")